# 🌐 Web Application Overview

This notebook walks through the usage and design of the Dash-based hotel prediction system. It predicts:
- **Cancellation Probability**
- **Average Daily Rate (ADR)**

### How model is used

#### User Inputs

In [70]:
from datetime import date

hotel_type = 'City Hotel' # City Hotel, Resort Hotel
room_type = 'A' # (A)Superior Room, (B)Standard Room, (C)Family Room, (D)Dulex Room, (E)Double Room, (F)Executive Room, (G)Premium Suite, (H)Junior Suite
checkin_date = '2025-4-12'
checkout_date = '2025-4-29'
adults = 2
children = 2
babies = 0
previous_cancellations = 0
meal = 'HB' # (BB)Bed and Breakfast, (HB)Half Board, (FB)Fuul Borad, (SC)Self Catering
special_requests = 2
market_segment = 'Online TA' # Direct, Corporate, Groups, Online TA, Offline TA/TO, Aviation
distribution_channel = 'TA/TO' # Direct, Corporate, TA/TO, GDS
repeated_guest = 0 # Yes(1), No(0)
customer_type = 'Transient' # Transient, Transient-Party, Contract, Group
deposit_type = 'Non Refund' # No Deposit, Refundable, Non Refund

#### Importing libraries, Loading models and encoders

In [71]:
import joblib
import pandas as pd
import numpy as np

# Load models
model_cancel = joblib.load("../artifacts/model_cancel.pkl")
model_adr = joblib.load("../artifacts/model_adr.pkl")

# Load encoder
encoders = joblib.load("../artifacts/encoders.pkl")
feature_order = joblib.load("../artifacts/adr_features.pkl")

#### Calculations and Processing

In [72]:
import datetime

date_obj = datetime.datetime.strptime(checkin_date, "%Y-%m-%d").date()
#calculate the lead time
today = datetime.date.today()
lead_time = (date_obj - today).days

# Calculate Week Nights and Weekend Nights based on the Check-in and Check-out Dates
checkin = datetime.datetime.strptime(checkin_date, "%Y-%m-%d")
checkout = datetime.datetime.strptime(checkout_date, "%Y-%m-%d")

nights = (checkout - checkin).days # Total nights
week_nights = 0
weekend_nights = 0

for i in range(nights):
    day = (checkin + datetime.timedelta(days=i)).weekday()
    if day < 5:  # Mon–Fri (0–4)
        week_nights += 1
    else:  # Sat–Sun (5–6)
        weekend_nights += 1

# Create input for ADR model (manually encoded as used during training)
hotel_enc = 1 if hotel_type == "Resort Hotel" else 0
deposit_enc = {"No Deposit": 0, "Refundable": 1, "Non Refund": 2}.get(deposit_type, 0)

#### Predictions

In [73]:
# Cancellation Prediction
cancel_input = pd.DataFrame([{
    'hotel': hotel_type,
    'lead_time': lead_time,
    # Dividing the Date from the DateTime Picker to these three (Month, Week and Day).
    'arrival_date_month': date_obj.strftime("%B"), # Convert from a month number to a month string (e.g 4 -> April)
    'arrival_date_week_number': date_obj.isocalendar().week,
    'arrival_date_day_of_month': date_obj.day,
    #----------------------------------------------------------------------------------------------------
    'stays_in_weekend_nights': weekend_nights,
    'stays_in_week_nights': week_nights,
    'adults': adults,
    'children': children,
    'babies': babies,
    'meal': meal,
    'market_segment': market_segment,
    'distribution_channel': distribution_channel,
    'is_repeated_guest': repeated_guest,
    'previous_cancellations': previous_cancellations,
    'previous_bookings_not_canceled': 0,
    'reserved_room_type': room_type,
    'assigned_room_type': room_type,
    'booking_changes': 0,
    'deposit_type': deposit_type,
    'agent': 0.0,
    'days_in_waiting_list': 0,
    'customer_type': customer_type,
    'required_car_parking_spaces': 0,
    'total_of_special_requests': special_requests
}])

# Prediction
cancel_prob = model_cancel.predict_proba(cancel_input)[0][1]

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ADR Prediction
adr_input = pd.DataFrame([{
    'hotel': hotel_enc,
    'lead_time': lead_time,
    'stays_in_weekend_nights': weekend_nights,
    'stays_in_week_nights': week_nights,
    'adults': adults,
    'children': children,
    'babies': babies,
    'meal': meal,
    'market_segment': market_segment,
    'distribution_channel': distribution_channel,
    'is_repeated_guest': repeated_guest,
    'previous_cancellations': previous_cancellations,
    'reserved_room_type': room_type,
    'assigned_room_type': room_type,
    'deposit_type': deposit_enc,
    'customer_type': customer_type,
    'total_of_special_requests': special_requests
}])

# Loading the encoders
label_encoders = joblib.load("../artifacts/encoders.pkl")
adr_feature_order = joblib.load("../artifacts/adr_features.pkl")

for col in ['meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'customer_type']:
    adr_input[col] = label_encoders[col].transform(adr_input[col])

# Reorder to match training
adr_input = adr_input[adr_feature_order]

# Prediction
adr_pred = np.expm1(model_adr.predict(adr_input)[0])

#### Output

In [74]:
# Output
print(f"Cancellation Probability: {cancel_prob * 100:.0f}%")
print(f"Estimated ADR: R{adr_pred:.2f}")

Cancellation Probability: 76%
Estimated ADR: R147.39
